In [150]:
import pandas as pd #importing libraries
import os
from datetime import datetime, timedelta

In [224]:
def add_rolling_time(source_dir, target):
    for EDA in os.listdir(source_dir): #for EDA files in the folder:
        if "EDA" in EDA: #if its have an "EDA" phrase:
            file_path = os.path.join(source_dir, EDA) #make a directory to this file
            excel = pd.read_csv(file_path) #read it
            GMT_date = pd.to_datetime((EDA.split("_")[2]+"250"), unit='ms') #make a GMT date by the given unix code of the file
            excel.loc[2,"real date"] = GMT_date + timedelta(hours=3) # add to the GMT date 3 hours (Israel time) and put it in new "real date" column - next to the first measurment
            for i in range (3,len(excel)): #from the third line to the end of the excel file:
                excel.loc[i,"real date"] = excel.loc[i-1,"real date"]+ timedelta(milliseconds=250) #to every following cell add 250 milliseconds (the EDA measure phase is 4Hz every second)
            target_path = os.path.join(target, EDA) #define the target path
            excel.to_csv(target_path)  #write the new file

In [225]:
def add_delta_time(new_source,target):
    for EDA in os.listdir(new_source): #for EDA files in the folder:
        file_path = os.path.join(new_source, EDA) #make a new path
        time_excel = pd.read_csv(file_path) #read the file from the new path
        delta_time = datetime.strptime(time_excel.loc[len(time_excel)-1,"real date"], "%Y-%m-%d %H:%M:%S.%f") -  datetime.strptime(time_excel.loc[3,"real date"], "%Y-%m-%d %H:%M:%S.%f") #calculate the duration time of the experiment
        time_excel.loc[0,"duration (minutes, seconds)"]  = delta_time #add it to a new column
        #divmod(delta_time.seconds, 60) ## if we want to convert it from "Time delta" type to "Tuple"
        target_path = os.path.join(target, EDA) #define the target path
        time_excel.to_csv(target_path)  #write the new file

In [226]:
def update_new_notes(new_source):
    for EDA in os.listdir(new_source): #for EDA files in the folder:
        file_path = os.path.join(new_source, EDA) #make a new path
        time_excel = pd.read_csv(file_path) #read the file from the new path
        Key = int(EDA.split("_")[0]) #Take its Primery Key 
        for index, row in enumerate(new_notes["Subject number"]): #for each row in new_notes
            if row == Key: #check if its subject number equals to the file primery key, if it does:
                new_notes.loc[index, "Real beginning time"] = time_excel.loc[2,"real date"] #add to that row the real beginning tima and,
                new_notes.loc[index, "Real end time"] = time_excel.loc[len(time_excel)-1,"real date"] #real ending time
                new_notes.loc[index,"duration"] = time_excel.loc[0,"duration (minutes, seconds)"] # and duration of the experiment.

In [ ]:
new_notes =pd.read_excel( "Path\\new_notes.xlsx") #read the new notes file

In [ ]:
source_dir = "Source_dir" #where the extracted files
target = 'Target_dir'#where to write the new files

In [228]:
add_rolling_time(source_dir, target) #adding to the EDA files the rolling experiment time

In [229]:
new_source = "Same as:'Target_dir'" #the path of the EDA files with the rolling time
add_delta_time(new_source,target) #adding to the EDA files the experiment duration

In [230]:
update_new_notes(new_source) #apdate the main table

In [244]:
new_notes.to_excel( "Path\\new_notes_1.xlsx") #write the apdated main table (with duration, real beginning and ending of the experiment)